In [60]:
import pandas as pd
import numpy as np
import os

In [61]:
df = pd.read_csv('../data/race_data/race_data.csv')

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65523 entries, 0 to 65522
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   RaceIndex         65523 non-null  int64 
 1   Pla.              65523 non-null  object
 2   Date              65523 non-null  object
 3   RC/Track/Course   65523 non-null  object
 4   Dist.             65523 non-null  int64 
 5   G                 65523 non-null  object
 6   RaceClass         65523 non-null  object
 7   Dr.               65523 non-null  object
 8   Rtg.              65523 non-null  object
 9   Trainer           65523 non-null  object
 10  Jockey            65474 non-null  object
 11  LBW               65523 non-null  object
 12  Win Odds          65523 non-null  object
 13  Act.Wt.           65523 non-null  int64 
 14  RunningPosition   65523 non-null  object
 15  Finish Time       65523 non-null  object
 16  Declar.Horse Wt.  65523 non-null  object
 17  Gear        

# Basic Cleaning

## utility function

In [63]:
# combine race index with date to form new index
def combine_index_date(df, col1, col2, new_col):

    df['race_index'] = df[col1].astype(str) + df[col2].dt.strftime('%d%m%y')

    df = df.drop(columns = ['RaceIndex'])

    return df


In [64]:
# convert to new race index format
def convert_year(date_str):
    day, month, year = date_str.split('/')

    if len(year) == 4:
        year = year[2:]

    date = f'{day}/{month}/{year}'
    
    return pd.to_datetime(date, format = '%d/%m/%y')

In [65]:
# clean RC, Track, Course
def clean_rc_track_course(text):
    parts = str(text).split('/')

    if len(parts) < 3:
        rc = parts[0].strip()
        track = parts[1].strip()
        course = None
    
    else:
        rc, track, course = parts[0].strip(), parts[1].strip(), parts[2].strip().replace('"', '')

    return rc, track, course

In [66]:
# clean origin, age
def clean_origin_age(text):
    parts = str(text).split('/')

    if len(parts) < 2:
        origin = parts[0].strip()
        age = None
    
    else:
        origin = parts[0].strip()
        age = parts[1].strip()
    
    return origin, age

In [67]:
# clean colour, sex
def clean_colour_sex(text):
    parts = str(text).split('/')

    colour = parts[0].strip()
    sex = parts[-1].strip()

    return colour, sex

## Workflow

In [68]:
df['Date'] = df['Date'].apply(convert_year)

In [69]:
df = combine_index_date(df, 'RaceIndex', 'Date', 'race_index')

In [70]:
df.groupby('race_index')['Pla.'].head()

0        12
1        05
2        02
3        11
4        05
         ..
62304    06
62485    12
62716    04
62896    07
63005    04
Name: Pla., Length: 30781, dtype: object

In [71]:
# exponential decay for top finishers
df['target'] = df.groupby('race_index')['Pla.'].transform(
    lambda x: x.apply(lambda place: np.exp(-(int(place)-1)/2) if str(place).isdigit() and int(place) <= 4 else 0)
)

In [72]:
df[['rc', 'track', 'course']] = df['RC/Track/Course'].apply(clean_rc_track_course).apply(pd.Series)
df = df.drop(columns=['RC/Track/Course'])

In [73]:
df[['origin', 'age']] = df['Origin / Age'].apply(clean_origin_age).apply(pd.Series)
df = df.drop(columns = ['Origin / Age'])

In [74]:
# clean colour, sex
df[['colour', 'sex']] = df['Colour / Sex'].apply(clean_colour_sex).apply(pd.Series)
df = df.drop(columns = ['Colour / Sex'])

In [75]:
# clean rating
df['Rtg.'] = pd.to_numeric(df['Rtg.'], errors = 'coerce')

In [76]:
# rename columns
df = df.rename(columns = {'Dr.' : 'gate_position', 'G' : 'track_condition'})

In [77]:
df = df[['Date', 'race_index', 'RaceClass', 'rc', 'track', 'course', 'Dist.', 'track_condition', 'Horse_id', 'Declar.Horse Wt.', 'Act.Wt.', 'gate_position', 'Rtg.', 'age', 'colour', 'sex', 'origin', 'Import type', 'Trainer', 'Jockey', 'Sire', 'Dam', 'Dam sire', 'Finish Time', 'Gear', 'target']]

In [78]:
for col in ['race_index', 'Declar.Horse Wt.', 'age']:
    df[col] = pd.to_numeric(df[col], errors = 'coerce')

In [79]:
df = df[df['Date'] > pd.to_datetime('04/09/20', format = '%d/%m/%y')]

In [80]:
df

,Date,race_index,RaceClass,rc,track,course,Dist.,track_condition,Horse_id,Declar.Horse Wt.,...,origin,Import type,Trainer,Jockey,Sire,Dam,Dam sire,Finish Time,Gear,target
0,2023-12-10,238101223,G1,ST,Turf,A,1600,G,H811,1187.0,...,JPN,VIS,T Yasuda,Y Kitamura,Just A Way,Epic Love,Dansili,1.35.46,H,0.000000
1,2023-04-30,623300423,G1,ST,Turf,A,2000,G,H811,1179.0,...,JPN,VIS,T Yasuda,C Y Ho,Just A Way,Epic Love,Dansili,2.02.71,H,0.000000
2,2022-12-11,240111222,G1,ST,Turf,A,2000,G,H811,1150.0,...,JPN,VIS,T Yasuda,Y Kitamura,Just A Way,Epic Love,Dansili,2.00.44,--,0.606531
3,2021-02-06,402060221,5,ST,Turf,C,1200,G,C017,1045.0,...,AUS,PPG,C H Yip,C Wong,Smart Missile,Pyrography,Danzero,1.10.66,CP-/TT-,0.000000
4,2020-12-26,296261220,5,ST,Turf,A+3,1200,G,C017,1058.0,...,AUS,PPG,C H Yip,M F Poon,Smart Missile,Pyrography,Danzero,1.10.42,CP/TT,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65518,2023-12-23,278231223,3,ST,Turf,C,1400,G,H459,1152.0,...,AUS,PP,C Fownes,C Y Ho,Impending,Isola Blu,Blackfriars,1.22.88,--,0.367879
65519,2023-12-10,240101223,3,ST,Turf,A,1200,G,H459,1153.0,...,AUS,PP,C Fownes,C Y Ho,Impending,Isola Blu,Blackfriars,1.09.52,--,0.606531
65520,2023-11-19,185191123,3,ST,Turf,B+2,1200,GF,H459,1155.0,...,AUS,PP,C Fownes,C Y Ho,Impending,Isola Blu,Blackfriars,1.08.94,--,0.606531
65521,2025-07-01,801010725,4,ST,Turf,C,1200,G,K334,1127.0,...,AUS,PPG,M Newnham,Z Purton,Street Boss,Varanasi,Encosta de Lago,1.10.15,B1,0.000000


# Feature engineering

In [81]:
df = df.sort_values(['Horse_id', 'Date'])

n = 3

df['recent_3_win_rate_horse'] = (
    df.groupby('Horse_id')['target']
    .transform(lambda x: x.shift().rolling(window=n, min_periods = 3).apply(lambda y: np.mean(y == 4),raw = True))
)

In [82]:
df = df.sort_values(['Jockey', 'Date'])

df['recent_3_win_rate_jockey'] = (
    df.groupby('Jockey')['target']
    .transform(lambda x: x.shift().rolling(window=n, min_periods = 3).apply(lambda y: np.mean(y == 4),raw = True))
)

In [83]:
# Expand your rolling statistics beyond just win rate
df['recent_5_avg_finish_pos'] = (
    df.groupby('Horse_id')['target']
    .transform(lambda x: x.shift().rolling(window=5, min_periods=2).mean())
)

df['recent_3_consistency'] = (
    df.groupby('Horse_id')['target']
    .transform(lambda x: x.shift().rolling(window=3, min_periods=2).std())
)

# Jockey-Trainer combination performance
df['jockey_trainer_combo_rate'] = (
    df.groupby(['Jockey', 'Trainer'])['target']
    .transform(lambda x: x.shift().expanding(min_periods=5).mean())
)

# Horse performance on specific track/distance combinations
df['horse_track_distance_rate'] = (
    df.groupby(['Horse_id', 'track', 'Dist.'])['target']
    .transform(lambda x: x.shift().expanding(min_periods=2).mean())
)

In [84]:
overall_mean_win_rate = np.mean(df['target'] == 4)
df['recent_3_win_rate_horse'] = df['recent_3_win_rate_horse'].fillna(overall_mean_win_rate)
df['recent_3_win_rate_jockey'] = df['recent_3_win_rate_jockey'].fillna(overall_mean_win_rate)
df['recent_5_avg_finish_pos'] = df['recent_5_avg_finish_pos'].fillna(overall_mean_win_rate)
df['recent_3_consistency'] = df['recent_3_consistency'].fillna(overall_mean_win_rate)
df['jockey_trainer_combo_rate'] = df['jockey_trainer_combo_rate'].fillna(overall_mean_win_rate)
df['horse_track_distance_rate'] = df['horse_track_distance_rate'].fillna(overall_mean_win_rate)

In [85]:
df.sort_values('race_index', ascending=False)

,Date,race_index,RaceClass,rc,track,course,Dist.,track_condition,Horse_id,Declar.Horse Wt.,...,Dam sire,Finish Time,Gear,target,recent_3_win_rate_horse,recent_3_win_rate_jockey,recent_5_avg_finish_pos,recent_3_consistency,jockey_trainer_combo_rate,horse_track_distance_rate
58494,2025-07-16,847160725,3,HV,Turf,B,1200,GF,J393,1035.0,...,Nayef,1.09.22,TT,0.367879,0.0,0.0,0.000000,0.000000,0.108834,0.000000
56454,2025-07-16,847160725,3,HV,Turf,B,1200,GF,J537,1176.0,...,Choisir,1.09.88,CP,0.000000,0.0,0.0,0.305783,0.128824,0.167728,0.305783
56168,2025-07-16,847160725,3,HV,Turf,B,1200,GF,K342,1076.0,...,Commands,1.10.30,--,0.000000,0.0,0.0,0.000000,0.000000,0.184535,0.000000
63166,2025-07-16,847160725,3,HV,Turf,B,1200,GF,J401,1120.0,...,Bernardini,1.09.53,CP,0.000000,0.0,0.0,0.000000,0.000000,0.084676,0.000000
62772,2025-07-16,847160725,3,HV,Turf,B,1200,GF,J396,1084.0,...,Duke Of Marmalade,1.09.26,TT,0.223130,0.0,0.0,0.273576,0.577350,0.122706,0.341970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16206,2021-09-05,1050921,5,ST,Turf,A,1400,G,D502,1191.0,...,Kitten's Joy,1.24.13,TT,0.000000,0.0,0.0,0.091970,0.212395,0.385333,0.183940
17238,2021-09-05,1050921,5,ST,Turf,A,1400,G,C046,1058.0,...,Vettori,1.23.27,--,0.000000,0.0,0.0,0.316188,0.305551,0.146308,0.000000
8400,2021-09-05,1050921,5,ST,Turf,A,1400,G,D190,1054.0,...,Green Desert,1.24.43,--,0.000000,0.0,0.0,0.000000,0.000000,0.061043,0.000000
42204,2021-09-05,1050921,5,ST,Turf,A,1400,G,A217,1091.0,...,Anabaa,1.23.22,B,0.000000,0.0,0.0,0.273576,0.000000,0.099228,0.000000


In [86]:
output_dir = '../data/'
output_path = os.path.join(output_dir, 'cleaned_data_new.csv')

os.makedirs(output_dir, exist_ok = True)

df.to_csv(output_path, index = False)